In [482]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import spacy
import nltk
nlp=spacy.load("en_core_web_lg")
pd.set_option("display.max_rows",None)
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\25bak\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [483]:
train=pd.read_csv("Data/train.csv")
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [484]:
test=pd.read_csv("Data/test.csv")
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [485]:
print("Null values in training set")
print(train.isnull().sum())
print("Null values in testing set")
print(test.isnull().sum())

Null values in training set
id             0
keyword       61
location    2533
text           0
target         0
dtype: int64
Null values in testing set
id             0
keyword       26
location    1105
text           0
dtype: int64


In [486]:
print(train[["keyword","text"]].sample(10))

          keyword                                               text
3528   eyewitness  DK Eyewitness Travel Guide: Denmark: travel gu...
5440       police  California police officer pulls gun on man wit...
5751         riot       @Live_Workshop selfie at booth or riot Kappa
6148        siren  @optich3cz #askH3cz i'm jealous now. Bc i want...
452        attack  ISIL claims suicide bombing at Saudi mosque th...
6256    snowstorm  @PrablematicLA @Adweek I'm actually currently ...
451        attack  it scares me that there's new versions of nucl...
1057  body%20bags  Fairfax investigating firefighter over Faceboo...
6252    snowstorm  I liked a @YouTube video http://t.co/z8Cp77lVz...
5608        razed  The Latest: More homes razed by Northern Calif...


In [487]:
train_real=train[train["target"]==1]
train_fake=train[train["target"]==0]

In [488]:
print(train_real.shape," is the size of real train dataset")
print(train_fake.shape," is the size of fake train dataset")

(3271, 5)  is the size of real train dataset
(4342, 5)  is the size of fake train dataset


In [489]:
print(train_real["keyword"].nunique(),train_fake["keyword"].nunique())
print(test["keyword"].nunique())
print(set(train.keyword.unique())-set(test.keyword.unique()))

220 218
221
set()


In [490]:
impWords=train["keyword"].unique()[:15]
print(impWords)

[nan 'ablaze' 'accident' 'aftershock' 'airplane%20accident' 'ambulance'
 'annihilated' 'annihilation' 'apocalypse' 'armageddon' 'army' 'arson'
 'arsonist' 'attack' 'attacked']


In [491]:
# plt.figure(figsize=(15,10))
# sns.countplot(x=train["keyword"],hue=train["target"])
# plt.xticks(rotation=90)

In [492]:
unique=train["keyword"].unique().astype("str")
# uniqueCollection=[unique[i:i+15] 
#                   for i in range(0,len(unique),15)]
# plt.figure(figsize=(10,15))
# subplot_counter=1
# for group in uniqueCollection:
#     plt.figure(figsize=(10,15))
#     plt.subplot(len(uniqueCollection),1,subplot_counter)
#     countplot=sns.countplot(data=train[train["keyword"].isin(group)],x="keyword",hue="target")
#     plt.xticks(rotation=90)
#     plt.legend(title="target",loc="lower left")

In [493]:
keywordsInput=train[train["keyword"].isna()]
keywordsInput.shape

(61, 5)

In [494]:
keywordsInput.sample(20)

,id,keyword,location,text,target
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7586,10840,NaN,NaN,#Sismo DETECTADO #JapÌ_n 15:41:07 Seismic inte...,1
7588,10842,NaN,NaN,BREAKING: #ISIS claims responsibility for mosq...,1
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7607,10867,NaN,NaN,#stormchase Violent Record Breaking EF-5 El Re...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
18,26,NaN,NaN,My car is so fast,0
30,44,NaN,NaN,The end!,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
13,19,NaN,NaN,#Flood in Bago Myanmar #We arrived Bago,1


In [495]:
unique.tolist()

['nan',
 'ablaze',
 'accident',
 'aftershock',
 'airplane%20accident',
 'ambulance',
 'annihilated',
 'annihilation',
 'apocalypse',
 'armageddon',
 'army',
 'arson',
 'arsonist',
 'attack',
 'attacked',
 'avalanche',
 'battle',
 'bioterror',
 'bioterrorism',
 'blaze',
 'blazing',
 'bleeding',
 'blew%20up',
 'blight',
 'blizzard',
 'blood',
 'bloody',
 'blown%20up',
 'body%20bag',
 'body%20bagging',
 'body%20bags',
 'bomb',
 'bombed',
 'bombing',
 'bridge%20collapse',
 'buildings%20burning',
 'buildings%20on%20fire',
 'burned',
 'burning',
 'burning%20buildings',
 'bush%20fires',
 'casualties',
 'casualty',
 'catastrophe',
 'catastrophic',
 'chemical%20emergency',
 'cliff%20fall',
 'collapse',
 'collapsed',
 'collide',
 'collided',
 'collision',
 'crash',
 'crashed',
 'crush',
 'crushed',
 'curfew',
 'cyclone',
 'damage',
 'danger',
 'dead',
 'death',
 'deaths',
 'debris',
 'deluge',
 'deluged',
 'demolish',
 'demolished',
 'demolition',
 'derail',
 'derailed',
 'derailment',
 'desolat

In [496]:
def keywordIn(text):
    
    for token in text.split():
        if str(token) in unique:
            return token
        else:
            continue
    return np.nan

In [497]:
# for i in unique:
#     if str(i).startswith("f"):
#         print(i)

In [498]:
text=keywordsInput["text"][1]
print(text)
print(keywordIn(text))

Forest fire near La Ronge Sask. Canada
fire


In [499]:
keywordsInput.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [500]:
keywordsInput["keyword"]=keywordsInput["text"].apply(keywordIn)

C:\Users\25bak\AppData\Local\Temp\ipykernel_14188\1018093707.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keywordsInput["keyword"]=keywordsInput["text"].apply(keywordIn)


In [501]:
keywordsInput.isna().sum()

id           0
keyword     39
location    61
text         0
target       0
dtype: int64

In [502]:
keywordsInput=keywordsInput[keywordsInput["keyword"].isna()]
    

In [503]:
# def lastKeyIn(text):
#     doc=nlp(text)
#     words=[]
#     for word in doc:
#         if word.is_stop or word.is_oov or word.is_space or word.is_punct:
#              pass
#         elif word.is_ascii:
#             if word.text in unique or word.lemma_ in unique:
#                 return word
#             words.append(word)
#     return words

In [504]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiment=SentimentIntensityAnalyzer()

In [505]:
def lastKeyIn(text):
    doc=nlp(text)
    high_neg=0
    negative_word=None
    for word in doc:
        if word.is_stop or word.is_oov or word.is_space or word.is_punct:
             pass
        elif word.is_ascii:
            if word.text in unique or word.lemma_ in unique:
                return word
            neg=sentiment.polarity_scores(word.text)["neg"]
            if neg>high_neg:
                  high_neg=neg
                  negative_word=word
    if high_neg<0.5:
         return("Others")
    return negative_word

In [506]:
# sentence="Forest Fire in the tornado and there was a crash course"
# print(lastKeyIn(sentence))

tornado


In [507]:
keywordsInput["keyword"]=keywordsInput["text"].str.lower().apply(lastKeyIn)

In [518]:
keywordsInput["keyword"].value_counts()

Others        16
earthquake     1
warning        1
volcano        1
fires          1
tornado        1
suicide        1
evacuation     1
earthquake     1
collided       1
flood          1
detonated      1
typhoon        1
died           1
sirens         1
seismic        1
police         1
explode        1
shit           1
ridiculous     1
flood          1
flooding       1
flooded        1
razed          1
Name: keyword, dtype: int64

In [514]:
# def keywordMaterial(text):
#     high_neg=0
#     negative_word=None
#     for word in text:
#             neg=sentiment.polarity_scores(word)["neg"]
#             if neg>high_neg:
#                   high_neg=neg
#                   negative_word=word
#     if high_neg<0.5:
#           return("Others")
    
#     return negative_word
# def lastKeyIn(text):
#     doc=nlp(text)
#     high_neg=0
#     negative_word=None
#     for word in doc:
#         if word.is_stop or word.is_oov or word.is_space or word.is_punct:
#              pass
#         elif word.is_ascii:
#             if word.text in unique or word.lemma_ in unique:
#                 return word
#             neg=sentiment.polarity_scores(word)["neg"]
#             if neg>high_neg:
#                   high_neg=neg
#                   negative_word=word
#     if high_neg<0.5:
#          return("Others")
#     return negative_word
        